In [45]:
import random
import pandas as pd
import numpy as np
import os

# Funciones útiles
En la siguiente sección se encuentran funciones útiles tanto para simular el juego, como para programar el desarrollador

## Funciones simulación juego

In [2]:
def evaluar_numero(guess, target):
    '''
    esta función evalúa cuántos puntos y famas obtiene
    el jugador dado su adivinación "guess" y el 
    objetivo "target"
    '''
    guess_num = str(guess)
    target_num = str(target)
    n = len(target_num)
    puntos = 0
    famas = 0
    for i in range(n):
        if guess_num[i] in target_num:
            puntos += 1
            
        if guess_num[i] == target_num[i]:
            famas += 1
            puntos -= 1
    
    return puntos, famas
    
def print_puntos_famas(puntos, famas):
    '''
    Esta función imprime los puntos y las famas dados
    como "*" para puntos y "f" para famas
    '''
    return "*"*puntos + "f"*famas

def generar_numero_aleatorio(n=4):
    '''
    Genera un número aleatorio de n cifras sin repetir
    entrega un string con tal número
    '''
    nums = [str(x) for x in range(10)]
    randoms = random.sample(nums, n)
    random_num = "".join(randoms)
    return random_num

In [5]:
def jugar_punto_y_fama(n=4, target = "random"):
    """
    Simular el juego de punto y fama.
    La consola le solicitará al usuario sus adivinaciones
    Si la adivinación es 'exit', 'end' o 'finish' el juego termina
    Si la adivinación es 'target' o 'show' se imprimirá el target. 
    
    Parameters:
    -----------
    n : int
        el número de dígitos del número a adivinar (default is False)
    target : str
        el valor objetivo del juego. si "random" se generará un valor aleatorio
        o el usuario puede pasar el propio target
    
    """
    if target == "random":
        target =generar_numero_aleatorio(n)
        
    guessed = False
    intentos = 0
    while not guessed:

        guess_num = input("ingrese su número: ")
        
        # special cases in the input to finish the game or show the target
        if guess_num in ["exit", "end", "finish"]:
            print(f"saliendo el juego en {intentos} intentos")
            break
        if guess_num in ["target", "show"]:
            print(f"el número a encontrar es {target}")
            continue
        
        intentos += 1
        puntos, famas = evaluar_numero(guess_num, target)
        print(f"[{intentos}]",guess_num, print_puntos_famas(puntos, famas))
        if famas == n:
            print(f"Felicidades, encontró el número en {intentos} intentos")
            guessed = True

## Funciones Solucionador
En la siguiente sección encontrará funciones necesarias para programar el solucionador. 
Estas funciones se basan en tomar una lista de posibles soluciones y eliminar o mantener sólo, las opciones que lógicamente debe descartar o mantener.

### Funciones para desechar/filtrar/mantener opciones deseadas

In [29]:
# Delete all rows that contains a number k
def delete_allcomb_containing_k(possibles, k):
    # Delete all rows that contains a number k
    left_possibles = possibles[~possibles.combinacion.str.contains(k)]
    return left_possibles


def keep_allcomb_containing_k(possibles, k):
    # Delete all rows that dont contain a number k
    left_possibles = possibles[possibles.combinacion.str.contains(k)]
    return left_possibles


def delete_allcomb_containing_any_in_group(possibles, group):
    # delete all rows that contain any element from the group
    left_possibles = possibles.copy()
    for elem2delete in group:
        left_possibles = left_possibles[~left_possibles.combinacion.str.contains(elem2delete)]
    return left_possibles


def delete_allcomb_containing_all_in_group(possibles, group):
    # delete all rows that contain all elements from the group
    left_possibles = possibles.copy()
    len_group = len(group)
    left_possibles["num_contained"] = 0
    for elem in group:
        left_possibles['num_contained'] += left_possibles.combinacion.apply(lambda x: elem in x)+0        
    left_possibles = left_possibles[left_possibles.num_contained!=len_group]
    return left_possibles


def keep_allcomb_containing_any_in_group(possibles, group):
    # keep all rows that contain any elements from the group
    left_possibles = possibles.copy()
    len_group = len(group)
    left_possibles["num_contained"] = 0
    for elem in group:
        left_possibles['num_contained'] += left_possibles.combinacion.apply(lambda x: elem in x)+0        
    left_possibles = left_possibles[left_possibles.num_contained>0]
    return left_possibles
    
    
def keep_allcomb_containing_all_in_group(possibles, group):
    # remain all rows that contain all elements from the group
    left_possibles = possibles.copy()
    for elem2delete in group:
        left_possibles = left_possibles[left_possibles.combinacion.str.contains(elem2delete)]
    return left_possibles


def delete_allcomb_containing_ordered_guess(possibles, guess):
    # delete all cambinations that contains the elements in giess in the
    # same order. Used one a guess number returns just points. 
    left_possibles = possibles.copy()
    for i, elem in enumerate(guess):
        left_possibles = left_possibles[~(left_possibles.combinacion.str[i] == elem)]
    return left_possibles


def keep_combinations_containing_kpoints_from_group(possibles, group, k):
    # this function will count the number of elements from group, present in 
    # every combination of possibles. The combinations that have more than k
    # elements from group, are going to be deleted. The combinations that have
    # exactly k elements from group, will be kept and returned
    left_possibles = possibles.copy()
    left_possibles["num_contained"] = 0
    for elem in group:
        left_possibles['num_contained'] += left_possibles.combinacion.apply(lambda x: elem in x)+0        
    left_possibles = left_possibles[left_possibles.num_contained==k]
    return left_possibles


def delete_combination(possibles, combination):
    return possibles[possibles.combinacion!=combination]


def get_indexes(vec, target = 1):
    # this function returns a list containing all the indexes 
    # where vec[index] == target
    indexes = []
    for idx, elem in enumerate(vec):
        if elem==target:
            indexes.append(idx)
    return indexes


In [33]:
def get_binary_groups(n, num_ones):
    # esta función genera los grupos posibles de cómo 
    # combinar num_famas unos, en un grupo de n elementos. 
    # los demás elementos serán 0s.
    num_ceros = n-num_ones
    stuff = [0] * num_ceros + [1]*num_ones
    
    subsets = set()
    for permut in itertools.permutations(stuff, n):
        subsets.add(permut)
    return list(subsets)


def delete_combinations_containing_elems_in_positions(possibles, guess, group):
    # this function deletes the combinations that contains certain elements
    # in the given positions. The elements come from the parameter "guess"
    # the given positions come from the parameter 'group'. 
    # The elements where group is 1 will be deleted. 

    left_possibles = possibles.copy() 

    # generate a query to identify the elements that fit the conditions
    left_possibles['delete'] = 0
    queries = []
    one_indexes =get_indexes(group)
    for index in one_indexes:
        actual_query = f'num{index+1} == "{guess[index]}"'
        queries.append(actual_query)
    
    final_query = " & ".join(queries)
#     print(final_query)

    # Get the indexes of the elements that fit the conditions of the final_query
    indexes2delete = left_possibles.query(final_query).index.values
    left_possibles.loc[indexes2delete, 'delete'] = 1

    # delete the combinations that fit the conditions
    left_possibles = left_possibles[left_possibles['delete']==0]
    
    return left_possibles


def delete_fames(possibles, guess, num_famas):
    left_possibles = possibles.copy()
    n = len(guess)
    # cambiar el nombre de la siguiente por binary_groups en vez de sólo groups
    groups = get_binary_groups(n, num_famas+1)  # se borrarán las agrupaciones de num_famas+1   
    for group in groups:
        left_possibles = delete_combinations_containing_elems_in_positions(left_possibles, guess, group)
#         print(left_possibles.shape)
    return left_possibles


def delete_impossible_groups(possibles, guess, score):
    left_possibles = possibles.copy()
    n = len(guess)
    binary_groups = get_binary_groups(n, score+1)  # se borrarán las agrupaciones de num_famas+1   
    for binary_group in binary_groups:
        group = [guess[i] for i,x in enumerate(binary_group) if x==1]
        left_possibles = delete_allcomb_containing_all_in_group(left_possibles, group)
    return left_possibles


### Funciones para ejecutar diferentes estrategias
En esta sección encontrará distintas estrategias programadas. Esto simula las acciones que haría un jugador, dada una estrategia que se quiera representar

In [35]:
# REFACTORIZAR PARA BORRAR EL PARÁMETRO n EN ESTAS FUNCIONES

def get_random_guess(possibles, n=4, seed=23):
    """
    Esta función generará una adivinación aleatoria de todas las posibles
    
    Parameters:
    -----------
    possibles : Pandas DataFrame
        dataframe con todas las combinaciones posibles restantes
    
    n : int
        este parámetro no está teniendo efecto. REFACTOR Y BORRAR 
    
    seed : int
        semilla del aleatorio para reproducir los experimentos
    """
    random.seed(seed)
    return random.choice(possibles.combinacion.values)

def get_greedy_guess(possibles, n=4):
    """
    Esta función generará una adivinación dadas las posibles
    combinaciones en el dataframe possibles. 
    Identifica cuál es el número más posible en la primera posición
    y genera un resultado con ese número de primero. Luego identifica
    el más posible en la segunda posición, de los restantes, y así arma 
    toda la solución
    
    Parameters:
    -----------
    possibles : Pandas DataFrame
        dataframe con todas las combinaciones posibles restantes
    
    n : int
        este parámetro se puede deducir de possibles. REFACTOR Y BORRAR 
    
    """
    guess = ""
    possibles_filtered = possibles.copy()
    for i in range(n):
#         print(i)
        guess+=(possibles_filtered.groupby(f'num{i+1}')[['combinacion']].count().sort_values(by='combinacion', ascending=False).reset_index().iloc[0, 0])
        possibles_filtered = possibles_filtered[possibles_filtered[f'num{i+1}']==guess[i]]
    return guess

def get_guess(possibles, n, method='greedy'):
    if method == 'greedy':
        return get_greedy_guess(possibles, n)
    if method == 'random':
        return get_random_guess(possibles, n)



# Probar al solucionador
En la siguiente sección, el solucionador adivinará un número del usuario
Ideal para jugar con otros amigos para ver si la máquina resuelve antes que un amigo


In [109]:
n=7

In [110]:
# Leer el df_posibles de acuerdo al n
combinations_files_path = os.path.join('..', 'combinations_files')
combinations_files_path
combination_path = os.path.join(combinations_files_path, f"all_combinations_size_{n}.csv")
combination_path

'..\\combinations_files\\all_combinations_size_7.csv'

In [111]:
df_all_combinations = pd.read_csv(combination_path, sep=';')
df_all_combinations.shape

(604800, 8)

In [112]:
# transformar todas las columnas a object. Como leyó números
# las trae numéricas por defecto
for col in df_all_combinations:
    df_all_combinations[col] = df_all_combinations[col].astype(str)
df_all_combinations.dtypes

combinacion    object
num1           object
num2           object
num3           object
num4           object
num5           object
num6           object
num7           object
dtype: object

In [113]:
# transformación para recuperar el 0 del frente
df_all_combinations['combinacion'] = df_all_combinations['combinacion'].apply(lambda x: '0'+x if len(x)<n else x)

In [114]:
df_posibles = df_all_combinations.copy()
df_posibles.shape

(604800, 8)

In [115]:
# transformación para recuperar el 0 del frente
df_posibles['combinacion'] = df_posibles['combinacion'].astype('str')
df_posibles['combinacion'] = df_posibles['combinacion'].apply(lambda x: '0'+x if len(x)<n else x)

In [116]:
guessed = False
intentos = 0


while not guessed:
    actual_guess = get_greedy_guess(df_posibles, n)
    print(actual_guess)
    
    intentos += 1
    puntos, famas = [int(x) for x in input('ingrese puntos y famas separados por espacio ').split()]#evaluar_numero(actual_guess, target)
    
    
    print(f"[{intentos}]",actual_guess, print_puntos_famas(puntos, famas))
    if famas == n:
        print(f"Felicidades, encontró el número en {intentos} intentos")
        guessed = True
        break
    
    score = puntos+famas # los dos puntos recibidos

    df_posibles = keep_combinations_containing_kpoints_from_group(df_posibles, actual_guess, score)
    print(df_posibles.shape, "keep combinations containing kpoints")

    # delete according to the number of fames
    df_posibles = delete_fames(df_posibles, actual_guess, famas)
    print(df_posibles.shape, "se quitaron elementos de acuerdo al número de famas")
    
#     if famas == 0:
#         df_posibles = delete_allcomb_containing_ordered_guess(df_posibles, actual_guess)
#         print(df_posibles.shape, "se quitó adivinación ordenada por 0 famas")
    if score == n:
        df_posibles = delete_combination(df_posibles, actual_guess)
    
    
    if famas == n:
        print(f"Felicidades, encontró el número en {intentos} intentos")
        guessed = True
        break

0123456
ingrese puntos y famas separados por espacio 3 2
[1] 0123456 ***ff
(317520, 9) keep combinations containing kpoints
(307314, 10) se quitaron elementos de acuerdo al número de famas
1270384
ingrese puntos y famas separados por espacio 5 0
[2] 1270384 *****
(146340, 10) keep combinations containing kpoints
(69694, 10) se quitaron elementos de acuerdo al número de famas
2394705
ingrese puntos y famas separados por espacio 4 0
[3] 2394705 ****
(23273, 10) keep combinations containing kpoints
(11845, 10) se quitaron elementos de acuerdo al número de famas
8641092
ingrese puntos y famas separados por espacio 2 2
[4] 8641092 **ff
(3564, 10) keep combinations containing kpoints
(3481, 10) se quitaron elementos de acuerdo al número de famas
3758160
ingrese puntos y famas separados por espacio 7 0
[5] 3758160 *******
(1173, 10) keep combinations containing kpoints
(325, 10) se quitaron elementos de acuerdo al número de famas
7081536
ingrese puntos y famas separados por espacio 6 1
[6] 70

In [ ]:
"""
## REFACTORIZAR EL CÓDIGO ANTERIOR:

1. Que sea una función que puedo ejecutar

2. Que no saque error si el usuario ingresa algo equivocado
que en vez de ello, lo mencione en el mensaje y ya.

"""
